# Processing Big Data - Data Ingestion
© Explore Data Science Academy

## Honour Code
I {**NELSON**, **MWEMBE**}, confirm - by submitting this document - that the solutions in this notebook are a result of my own work and that I abide by the [EDSA honour code](https://drive.google.com/file/d/1QDCjGZJ8-FmJE3bZdIQNwnJyQKPhHZBn/view?usp=sharing).
    Non-compliance with the honour code constitutes a material breach of contract.



## Context 

To work constructively with any dataset, one needs to create an ingestion profile to make sure that the data at the source can be readily consumed. For this section of the predict, as the Data Engineer in the team, you will be required to design and implement the ingestion process. For the purposes of the project the AWS cloud storage service, namely, the S3 bucket service will act as your data source. All the data required can be found [here](https://processing-big-data-predict-stocks-data.s3.eu-west-1.amazonaws.com/stocks.zip).

<div align="center" style="width: 600px; font-size: 80%; text-align: center; margin: 0 auto">
<img src="https://raw.githubusercontent.com/Explore-AI/Pictures/master/data_engineering/transform/predict/DataIngestion.jpg"
     alt="Data Ingestion"
     style="float: center; padding-bottom=0.5em"
     width=40%/>
     <p><em>Figure 1. Data Ingestion</em></p>
</div>

Your manager, Gnissecorp Atadgib, knowing very well that you've recently completed your Data Engineering qualification, asks you to make use of Apache Spark for the ingestion as well as the rest of the project. His rationale being, that stock market data is generated every day and is quite time-sensitive and would require scalability when deploying to a production environment. 

## Dataset - US Nasdaq




<div align="center" style="width: 600px; font-size: 80%; text-align: center; margin: 0 auto">
<img src="https://raw.githubusercontent.com/Explore-AI/Pictures/master/data_engineering/transform/predict/Nasdaq.png"
     alt="Nasdaq"
     style="float: center; padding-bottom=0.5em"
     width=50%/>
     <p><em>Figure 2. Nasdaq</em></p>
</div>

The data that you will be working with is a historical snapshot of market data taken from the Nasdaq electronic market. This dataset contains historical daily prices for all tickers currently trading on Nasdaq. The up-to-date list can be found on their [website](https://www.nasdaq.com/)


The provided data contains price data dating back from 02 January 1962 up until 01 April 2020. The data found in the S3 bucket has been stored in the following structure:

```
     stocks/<Year>/<Month>/<Day>/stocks.csv
```
Each CSV file for every trading day contains the following details:
- **Date** - specifies trading date
- **Open** - opening price
- **High** - maximum price during the day
- **Low** - minimum price during the day
- **Close** - close price adjusted for splits
- **Adj Close** - close price adjusted for both dividends and splits
- **Volume** - the number of shares that changed hands during a given day

## Basic initialisation
To get you started, let's import some basic Python libraries as well as Spark modules and functions.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *


Remember that we need a `SparkContext` and `SparkSession` to interface with Spark.
We will mostly be using the `SparkContext` to interact with RDDs and the `SparkSession` to interface with Python objects.

> ℹ️ **Instructions** ℹ️
>
>Initialise a new **Spark Context** and **Session** that you will use to interface with Spark.

In [2]:
#TODO: Write your code here
sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

## Investigate dataset schema
At this point, it is enough to read in a single file to ascertain the data structure. You will be required to use the information obtained from the small subset to create a data schema. This data schema will be used when reading the entire dataset using Spark.

> ℹ️ **Instructions** ℹ️
>
>Make use of Pandas to read in a single file and investigate the plausible data types to be used when creating a Spark data schema. 
>
>*You may use as many coding cells as necessary.*

In [3]:
#TODO: Write your code here
file_path = "C:/Users/nmwem/Downloads/Compressed/processing-big-data-predict-main/Stocks/2020/03/02/stocks.csv"

df = pd.read_csv(file_path)

df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5841 entries, 0 to 5840
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       5841 non-null   object 
 1   Open       5841 non-null   float64
 2   High       5841 non-null   float64
 3   Low        5841 non-null   float64
 4   Close      5841 non-null   float64
 5   Adj Close  5841 non-null   float64
 6   Volume     5841 non-null   float64
 7   stock      5841 non-null   object 
dtypes: float64(6), object(2)
memory usage: 365.2+ KB


,Date,Open,High,Low,Close,Adj Close,Volume,stock
0,2020-03-02,77.309998,82.000000,77.239998,81.970001,81.761925,3407500.0,A
1,2020-03-02,14.020000,14.030000,13.110000,13.730000,13.730000,6784800.0,AA
2,2020-03-02,1.000000,1.000000,0.980000,0.980000,0.980000,13500.0,AACG
3,2020-03-02,19.049999,19.059999,17.510000,18.860001,18.860001,38738000.0,AAL
4,2020-03-02,20.799999,20.799999,20.799999,20.799999,20.799999,300.0,AAMC


## Read CSV files

When working with big data, it is often not tenable to keep processing an entire data batch when you are in the process of development - this can be quite time-consuming. If the data is uniform, it is sufficient to work with a smaller subset to create basic functionality. Your manager has identified the year **1962** to perform the initial testing for data ingestion. 

> ℹ️ **Instructions** ℹ️
>
>Read in the data for **1962** using a data schema that purely uses string data types. You will be required to convert to the appropriate data types at a later stage.
>
>*You may use as many coding cells as necessary.*

In [4]:
#TODO: Write your code here

file_path = "C:/Users/nmwem/Downloads/Compressed/processing-big-data-predict-main/Stocks/1962/*/*/*.csv"

schema = StructType([
    StructField("Date", StringType(), True),
    StructField("Open", StringType(), True),
    StructField("High", StringType(), True),
    StructField("Low", StringType(), True),
    StructField("Close", StringType(), True),
    StructField("Adj Close", StringType(), True),
    StructField("Volume", StringType(), True),
    StructField("stock", StringType(), True)
])

df = spark.read.csv(file_path, header=True, schema=schema)

df.show(10)


+----------+------------------+------------------+------------------+------------------+------------------+---------+-----+
|      Date|              Open|              High|               Low|             Close|         Adj Close|   Volume|stock|
+----------+------------------+------------------+------------------+------------------+------------------+---------+-----+
|1962-02-19| 5.839290142059326| 5.907374858856201| 5.839290142059326| 5.863319873809815|1.3863292932510376|  29900.0|   AA|
|1962-02-19| 5.481634140014648|5.5284857749938965| 5.481634140014648| 5.516772747039795|1.2804527282714844|  32000.0| ARNC|
|1962-02-19|0.9074074029922484|0.9156378507614136|0.8991769552230835| 0.903292179107666|0.1614154428243637| 619400.0|   BA|
|1962-02-19|1.6770833730697632|1.6927083730697632|1.6614583730697632|1.6770833730697632|0.1440587043762207| 170400.0|  CAT|
|1962-02-19|               0.0|3.5788691043853764|              20.0| 3.549107074737549|0.0565012246370315| 273600.0|  CVX|
|1962-02

## Update column names
To make the data easier to work with, you will need to make a few changes:
1. Column headers should all be in lowercase; and
2. Whitespaces should be replaced with underscores.


> ℹ️ **Instructions** ℹ️
>
>Make sure that the column headers are all in lowercase and that any whitespaces are replaced with underscores.
>
>*You may use as many coding cells as necessary.*

In [5]:
#TODO: Write your code here

df = df.toDF(*(c.lower().replace(" ", "_") for c in df.columns))

df.show(10)


+----------+------------------+------------------+------------------+------------------+------------------+---------+-----+
|      date|              open|              high|               low|             close|         adj_close|   volume|stock|
+----------+------------------+------------------+------------------+------------------+------------------+---------+-----+
|1962-02-19| 5.839290142059326| 5.907374858856201| 5.839290142059326| 5.863319873809815|1.3863292932510376|  29900.0|   AA|
|1962-02-19| 5.481634140014648|5.5284857749938965| 5.481634140014648| 5.516772747039795|1.2804527282714844|  32000.0| ARNC|
|1962-02-19|0.9074074029922484|0.9156378507614136|0.8991769552230835| 0.903292179107666|0.1614154428243637| 619400.0|   BA|
|1962-02-19|1.6770833730697632|1.6927083730697632|1.6614583730697632|1.6770833730697632|0.1440587043762207| 170400.0|  CAT|
|1962-02-19|               0.0|3.5788691043853764|              20.0| 3.549107074737549|0.0565012246370315| 273600.0|  CVX|
|1962-02

## Null Values
Null values often represent missing pieces of data. It is always good to know where your null values lie - so you can quickly identify and remedy any issues stemming from these.

> ℹ️ **Instructions** ℹ️
>
>Write code to count the number of null values found in each column.
>
>*You may use as many coding cells as necessary.*

In [6]:
#TODO: Write your code here

def count_nulls(df):
    null_counts = []
    for col in df.columns:
        null_count = df.filter(df[col].isNull()).count()
        null_counts.append((col, null_count))
    return null_counts

null_counts = count_nulls(df)
for col, null_count in null_counts:
    print(f"Column '{col}' has {null_count} null values.")


Column 'date' has 0 null values.
Column 'open' has 0 null values.
Column 'high' has 0 null values.
Column 'low' has 22 null values.
Column 'close' has 0 null values.
Column 'adj_close' has 0 null values.
Column 'volume' has 21 null values.
Column 'stock' has 0 null values.


In [7]:
def count_nulls(df):
    null_counts = []
    for col in df.columns:
        null_count = df.filter(df[col].isNull()).count()
        null_counts.append(null_count)
    return sum(null_counts)

pre_count = count_nulls(df)
print(f"The pre-conversion count of null values is {pre_count}.")


The pre-conversion count of null values is 43.


## Data type conversion - The final data schema

Now that we have identified the number of missing values in the data set, we'll move on to convert our data schema to the required data types. 

> ℹ️ **Instructions** ℹ️
>
>Use typecasting to convert the string data types in your current data schema to more appropriate data types.
>
>*You may use as many coding cells as necessary.*

In [8]:
df.printSchema()

root
 |-- date: string (nullable = true)
 |-- open: string (nullable = true)
 |-- high: string (nullable = true)
 |-- low: string (nullable = true)
 |-- close: string (nullable = true)
 |-- adj_close: string (nullable = true)
 |-- volume: string (nullable = true)
 |-- stock: string (nullable = true)



In [10]:
from pyspark.sql.types import StringType, StructType, StructField, IntegerType, FloatType

df = df.withColumn('date', F.to_timestamp(F.col('date')))\
    .withColumn("open", F.col("open").cast(FloatType()))\
    .withColumn("high", F.col("high").cast(FloatType()))\
    .withColumn("low", F.col("low").cast(FloatType()))\
    .withColumn("close", F.col("close").cast(FloatType()))\
    .withColumn("adj_close", F.col("adj_close").cast(FloatType()))\
    .withColumn("volume", F.col("volume").cast(FloatType()))\
    .withColumn("stock", F.col("stock").cast(StringType()))


In [11]:
df.printSchema()

root
 |-- date: timestamp (nullable = true)
 |-- open: float (nullable = true)
 |-- high: float (nullable = true)
 |-- low: float (nullable = true)
 |-- close: float (nullable = true)
 |-- adj_close: float (nullable = true)
 |-- volume: float (nullable = true)
 |-- stock: string (nullable = true)



## Consolidate missing values
We have to check if the data type conversion above was done correctly.
If the casting was not successful, a null value gets inserted into the dataframe. You can thus check for successful conversion by determining if any null values are included in the resulting dataframe.


> ℹ️ **Instructions** ℹ️
>
>Write code to compare the number of invalid entries (nulls) pre-conversion and post-conversion.
>
>*You may use as many coding cells as necessary.*

In [12]:
def count_nulls(df):
    null_counts = []
    for col in df.columns:
        null_count = df.filter(df[col].isNull()).count()
        null_counts.append(null_count)
    return sum(null_counts)

post_count = count_nulls(df)
print(f"The post-conversion count of null values is {post_count}.")



The post-conversion count of null values is 84.


In [13]:
if pre_count == post_count:
    print("casting was successful")
else:
    print("casting was not successful")



casting was not successful


Here you should be able to see if any of your casts went wrong. 
Do not attempt to correct any missing values at this point. This will be dealt with in later sections of the predict.

## Generate parquet files
When writing in Spark, we typically use parquet format. This format allows parallel writing using Spark's optimisation while maintaining other useful things like metadata.

When writing, it is good to make sure that the data is sufficiently partitioned. 

Generally, data should be partitioned with one partition for every 200MB of data, but this also depends on the size of your cluster and executors. 


### Check the size of the dataframe before partitioning

In [14]:
from pyspark.serializers import PickleSerializer, AutoBatchedSerializer

In [15]:
rdd = df.rdd._reserialize(AutoBatchedSerializer(PickleSerializer()))
obj = rdd.ctx._jvm.org.apache.spark.mllib.api.python.SerDe.pythonToJava(rdd._jrdd, True)
size = sc._jvm.org.apache.spark.util.SizeEstimator.estimate(obj)
size_MB = size/1000000
partitions = max(int(size_MB/200), 2)
print(f'The dataframe is {size_MB} MB')

The dataframe is 12.902696 MB


### Write parquet files to the local directory
> ℹ️ **Instructions** ℹ️
>
> Use the **coalesce** function and the number of **partitions** derived above to write parquet files to your local directory 
>
>*You may use as many coding cells as necessary.*

In [18]:
#TODO: Write your code here
df.coalesce(partitions).write.format("parquet").mode("append").save("C:/Users/nmwem/Downloads/Compressed/processing-big-data-predict-main/Task1_data_ingestion/first_output")

In [19]:
spark.stop()